## Importing libraries

In [4]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
from collections import OrderedDict
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca
# import os; os.chdir('/home/juan/activitysynth/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from sklearn.neighbors import BallTree

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs
import math 
from collections import OrderedDict 

import pickle
import dill
import time

import urbansim_templates

from scripts import datasources, models, variables, utils

## Preparing data for simulation

In [5]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [6]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [7]:
orca.run(['initialize_network_small']) 
orca.run([ 'initialize_network_walk']) 
orca.run(['initialize_imputed_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_imputed_skims'
No imputed skims found. Creating them now.
Time to execute step 'initialize_imputed_skims': 89.81 s
Total time to execute iteration 1 with iteration value None: 89.81 s


## Simulation 

In [8]:
t_start = time.time()

df = orca.get_table('long_format').to_frame()
file_Name = "/home/juan/activitysynth/activitysynth/configs/SLCM_gen_tt.pkl"
fileObject = open(file_Name,'rb')  
SLCM = pickle.load(fileObject)  

df['probabilities'] = SLCM.predict(df)
df.sort_values("probabilities", ascending=False).groupby('obs_id').agg({'school_choice_set': 'first'})

t_end = time.time()
t1 = t_end - t_start
print("Time: {:.2f} min".format(t1/60))

Time: 14.10 min


In [11]:
@orca.step()
def SLCM_simulate(long_format):
    """
    Generate school location choices for the synthetic pop households.
    
    """
    file_Name = "/home/juan/activitysynth/activitysynth/configs/SLCM_gen_tt.pkl"
    fileObject = open(file_Name,'rb')  
    SLCM = pickle.load(fileObject)
    
    orca.add_column('long_format', 'probabilities', SLCM.predict(orca.get_table('long_format').to_frame()))
#     df['probabilities'] = SLCM.predict(orca.get_table('long_format'))
    df = orca.get_table('long_format').to_frame()
    SL = df.sort_values("probabilities", ascending=False).groupby('obs_id').agg({'school_choice_set': 'first'})

In [12]:
orca.run(['SLCM_simulate']) 

Running step 'SLCM_simulate'
Time to execute step 'SLCM_simulate': 519.88 s
Total time to execute iteration 1 with iteration value None: 519.88 s


In [13]:
orca.eval_step()

NameError: name 'SL' is not defined